In [23]:
import pandas as pd
from sqlalchemy import create_engine

In [5]:
player_info = pd.read_csv("player_info.csv")
game_skater_stats = pd.read_csv("game_skater_stats.csv")

In [172]:
url = "https://puckpedia.com/players"
puckpedia_df = pd.read_html(url)[0]

In [189]:
puckpedia_df.head()

,Unnamed: 0,Unnamed: 1,Player,Team,Current Cap Hit,Current Salary,Current Cap %,Age,Height,Weight,...,2019-20 P,2019-20 GAA,2019-20 SV%,2019-20 GF%,2019-20 DFF%,2019-20 relDFF%,2019-20 CF%,2019-20 relCF%,2019-20 G60,2019-20 P60
0,1,NaN,Connor McDavid,NaN,"$12,500,000","$15,000,000",15.3%,23,73in,193lbs,...,97.0,NaN,NaN,51.3,49.6,-0.1,47.5,-0.4,1.20,2.86
1,2,NaN,Artemi Panarin,NaN,"$11,642,857","$14,000,000",14.3%,28,71in,170lbs,...,95.0,NaN,NaN,65.5,52.7,7.0,50.3,5.6,1.16,3.16
2,3,NaN,Auston Matthews,NaN,"$11,634,000","$15,900,000",14.3%,22,75in,220lbs,...,80.0,NaN,NaN,59.4,55.6,5.9,54.4,3.3,1.56,2.34
3,4,NaN,Erik Karlsson,NaN,"$11,500,000","$14,500,000",14.1%,29,72in,190lbs,...,40.0,NaN,NaN,48.8,50.2,2.2,52.8,3.9,0.24,1.51
4,5,NaN,John Tavares,NaN,"$11,000,000","$15,900,000",13.5%,29,73in,215lbs,...,60.0,NaN,NaN,45.5,52.7,0.7,53.3,1.3,0.80,1.79


In [182]:
# put players' names into two separate columns with first and last names separated
first_name = []
last_name = []
for name in puckpedia_df["Player"]:
    first_name.append(name.split(" ")[0])
    last_name.append(name.split(" ")[1])

In [184]:
current_salary = [salary for salary in puckpedia_df['Current Salary']]

In [210]:
# Create a new dataframe displaying the players' first and last names and their salaries
puckpedia_playersdf = pd.DataFrame({"first_name":first_name, "last_name":last_name, "current_salary": current_salary})

In [9]:
# Drop columns thaqt we don't need in our research
player_info_df = player_info.drop(columns = ["birthCity", "primaryPosition", "birthDate", "link"])


,player_id,firstName,lastName,nationality
0,8467412,Alexei,Ponikarovsky,UKR
1,8468501,Anton,Volchenkov,RUS
2,8459670,Kimmo,Timonen,FIN
3,8471233,Travis,Zajac,CAN
4,8455710,Martin,Brodeur,CAN


In [13]:
players_df.drop(columns=["penaltyMinutes", "faceOffWins", "blocked", "plusMinus", "evenTimeOnIce", "shortHandedTimeOnIce",\
                        "powerPlayTimeOnIce", "timeOnIce"], inplace=True)

In [193]:
players_df.drop(columns=["team_id"],inplace=True)

KeyError: "['team_id'] not found in axis"

In [199]:
players_df.head()

,game_id,player_id,assists,goals,shots,hits
0,2011030221,8467412,0,0,1,3
1,2011030221,8468501,0,0,0,4
2,2011030221,8470609,0,0,2,1
3,2011030221,8471816,0,0,1,4
4,2011030221,8472410,0,0,1,3


# Transform players_info and players_df dataframes

Transform players_info 

In [203]:
# Rename the columns
player_info_df.rename(columns={"firstName":"first_name",
                              "lastName": "last_name"}, inplace=True)
# Delete duplicate player_ids and set player+ids as index
player_info_df.drop_duplicates("player_id", inplace = True)
player_info_df.set_index("player_id", inplace=True)


Transform players_df

In [202]:
# Delete duplicate player_ids
players_df.drop_duplicates("player_id", inplace = True)
# Set index
players_df.set_index("player_id", inplace=True)

In [211]:
# Display all our dataframes for fun
print(player_info_df.head())
print(puckpedia_playersdf.head())
print(players_df.head())

          first_name     last_name nationality
player_id                                     
8467412       Alexei  Ponikarovsky         UKR
8468501        Anton    Volchenkov         RUS
8459670        Kimmo       Timonen         FIN
8471233       Travis         Zajac         CAN
8455710       Martin       Brodeur         CAN
  first_name last_name current_salary
0     Connor   McDavid    $15,000,000
1     Artemi   Panarin    $14,000,000
2     Auston  Matthews    $15,900,000
3       Erik  Karlsson    $14,500,000
4       John   Tavares    $15,900,000
              game_id  assists  goals  shots  hits
player_id                                         
8467412    2011030221        0      0      1     3
8468501    2011030221        0      0      0     4
8470609    2011030221        0      0      2     1
8471816    2011030221        0      0      1     4
8472410    2011030221        0      0      1     3


In [213]:
connection_string = "postgres:postgres@localhost:5432/nhl_db"
engine = create_engine(f'postgresql://{connection_string}')

In [214]:
engine.table_names()

['players', 'player_info', 'player_salaries']

In [205]:
players_df.to_sql(name="players", con=engine, if_exists="replace", index=True)

In [206]:
player_info_df.to_sql(name="player_info", con=engine, if_exists="replace", index=True)

In [212]:
puckpedia_playersdf.to_sql(name="player_salaries", con=engine, if_exists="replace", index=True)